<a href="https://colab.research.google.com/github/LosSantosDeDomingo/MagicWand/blob/main/Jose_Caraballo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Jose Caraballo
# Homework 2
# 02/16/2025

In [ ]:
# Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestNeighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# Functions
def centerData(dataset):
    mean = dataset.mean()
    centeredDataset = dataset - mean

    return centeredDataset


def calculateCovarianceMatrix(centeredDataset):
    dotProduct = np.dot(centeredDataset.T, centeredDataset)
    normalization = centeredDataset.shape[0] - 1
    covarianceMatrix = dotProduct / normalization

    return covarianceMatrix

def centerKernel(kernel):
    dummyElement = kernel.shape[0]
    dummyMatrix = np.ones((dummyElement, dummyElement)) / dummyElement
    meanRow = np.dot(dummyMatrix, kernel)
    meanColumn = np.dot(kernel, dummyMatrix)
    totalMean = np.dot(dummyMatrix , meanColumn)
    centeredKernel = kernel - meanRow - meanColumn + totalMean

    return centeredKernel

def eigenDecomposition(dataset):
    eigenvalues, eigenvectors = np.linalg.eig(dataset)
    sortedEigen = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sortedEigen]
    eigenvectors = eigenvectors[:,sortedEigen]

    return eigenvalues, eigenvectors

def performPCA(centeredDataset, desiredVarianceRatio, eigenvalues, eigenvectors):
    # Determine the Variance Ratio
    absEigenvalues = np.abs(eigenvalues)
    sumEigenvalues = np.sum(absEigenvalues)
    varianceRatio = absEigenvalues / sumEigenvalues
    cumulativeVarianceRatio = np.cumsum(varianceRatio)

    # Traverse the Cumulative Variance Ratio to determine the Desired Variance Ratio
    eigenvaluesCounter = 0

    for i in cumulativeVarianceRatio:
        eigenvaluesCounter += 1
        if i >= desiredVarianceRatio:
            foundVarianceRatio = i
            print("The Needed Amount of Eigenvalues:", eigenvaluesCounter)
            print("Appropriate Variance Ratio:", foundVarianceRatio)
            break

    PCA = np.dot(centeredDataset, eigenvectors[:,:eigenvaluesCounter])

    return PCA, eigenvaluesCounter, foundVarianceRatio

def weightedKernelSum(alpha, beta, kernelAlpha, kernelBeta):
   weightedSum = (alpha * kernelAlpha) + (beta * kernelBeta)

   return weightedSum

def kernelRBF(dataset):
    # Initialize Kernel
    # Error Handling for numpy
    if hasattr(dataset, 'to_numpy'):
        numpyDataset = dataset.to_numpy()
    else:
        numpyDataset = dataset

    dataPoints = dataset.shape[0]
    matrixRBF = np.zeros((dataPoints, dataPoints))
    features = dataset.shape[1]
    gamma = 1 / features

    # Create Kernel
    for i in range(dataPoints):
        for j in range(dataPoints):
            difference = numpyDataset[i] - numpyDataset[j]
            distanceSquared = np.dot(difference, difference)
            matrixRBF[i, j] = np.exp(-gamma * distanceSquared)

    return matrixRBF

def kernelPoly(dataset):
    # Initialize Kernel
    # Error Handling for numpy
    if hasattr(dataset, 'to_numpy'):
        numpyDataset = dataset.to_numpy()
    else:
        numpyDataset = dataset

    dataPoints = dataset.shape[0]
    matrixPoly = np.zeros((dataPoints, dataPoints))
    degree = 3
    coefficient = 1
    features = dataset.shape[1]
    gamma = 1 / features

    # Create Kernel
    for i in range(dataPoints):
        for j in range(dataPoints):
            dotProduct = np.dot(numpyDataset[i], numpyDataset[j])
            matrixPoly[i, j] = (gamma * dotProduct + coefficient) ** degree

    return matrixPoly

def kernelLinear(dataset):
    # Initialize Kernel
    # Error Handling for numpy
    if hasattr(dataset, 'to_numpy'):
        numpyDataset = dataset.to_numpy()
    else:
        numpyDataset = dataset

    dataPoints = dataset.shape[0]
    matrixLinear = np.zeros((dataPoints, dataPoints))
    numpyDataset = dataset.to_numpy()

    # Create Kernel
    for i in range(dataPoints):
        for j in range(dataPoints):
            dotProduct = np.dot(numpyDataset[i], numpyDataset[j])
            matrixLinear[i, j] = dotProduct
            matrixLinear[j, i] = dotProduct

    return matrixLinear

def crossKernelRBF(dataset1, dataset2):
    # Initialize Kernel
    # Error Handling for numpy
    if hasattr(dataset1, 'to_numpy'):
        numpyDataset1 = dataset1.to_numpy()
    else:
        numpyDataset1 = dataset1
    if hasattr(dataset2, 'to_numpy'):
        numpyDataset2 = dataset2.to_numpy()
    else:
        numpyDataset2 = dataset2

    dataPoint1 = numpyDataset1.shape[0]
    dataPoint2 = numpyDataset2.shape[0]
    features = numpyDataset1.shape[1]
    gamma = 1 / features
    crossMatrixRBF = np.zeros((dataPoint1, dataPoint2))

    # Create Kernel
    for i in range(dataPoint1):
        for j in range(dataPoint2):
            difference = numpyDataset1[i] - numpyDataset2[j]
            distanceSquared = np.dot(difference, difference)
            crossMatrixRBF[i, j] = np.exp(-gamma * distanceSquared)
    return crossMatrixRBF

def crossKernelPoly(dataset1, dataset2):
    # Initialize Kernel
    # Error Handling for numpy
    if hasattr(dataset1, 'to_numpy'):
        numpyDataset1 = dataset1.to_numpy()
    else:
        numpyDataset1 = dataset1
    if hasattr(dataset2, 'to_numpy'):
        numpyDataset2 = dataset2.to_numpy()
    else:
        numpyDataset2 = dataset2

    dataPoint1 = numpyDataset1.shape[0]
    dataPoint2 = numpyDataset2.shape[0]
    features = numpyDataset1.shape[1]
    gamma = 1 / features
    crossMatrixPoly = np.zeros((dataPoint1, dataPoint2))
    degree = 3
    coefficient = 1

    # Create Kernel
    for i in range(dataPoint1):
        for j in range(dataPoint2):
            dotProduct = np.dot(numpyDataset1[i], numpyDataset2[j])
            crossMatrixPoly[i, j] = (gamma * dotProduct + coefficient) ** degree

    return crossMatrixPoly

def crossKernelLinear(dataset1, dataset2):
    # Initialize Kernel
    # Error Handling for numpy
    if hasattr(dataset1, 'to_numpy'):
        numpyDataset1 = dataset1.to_numpy()
    else:
        numpyDataset1 = dataset1
    if hasattr(dataset2, 'to_numpy'):
        numpyDataset2 = dataset2.to_numpy()
    else:
        numpyDataset2 = dataset2

    dataPoint1 = numpyDataset1.shape[0]
    dataPoint2 = numpyDataset2.shape[0]
    crossMatrixLinear = np.zeros((dataPoint1, dataPoint2))

    # Create Kernel
    for i in range(dataPoint1):
        for j in range(dataPoint2):
            crossMatrixLinear[i, j] = np.dot(numpyDataset1[i], numpyDataset2[j])

    return crossMatrixLinear

def centerCrossKernel(testKernel, trainKernel):
    trainMean = np.mean(trainKernel)
    trainColumnMean = np.mean(trainKernel, axis=0)
    testRowMean = np.mean(testKernel, axis=1, keepdims=True)
    centeredTestKernel = testKernel - testRowMean - trainColumnMean + trainMean

    return centeredTestKernel

def accuracyMetric(yTest, yPredict):
    accuracyScore = 100 * accuracy_score(yTest,yPredict)

    return accuracyScore


def naiveGaussianClassifier(xTrain, xTest, yTrain, yTest):
    naiveGaussian = GaussianNB()
    naiveGaussian.fit(xTrain, yTrain)
    yPredict = naiveGaussian.predict(xTest)

    return xTrain, xTest, yTrain, yTest, yPredict

def calculateTopFeatures(dataFrame, featureAmount):
    centeredDataFrame = centerData(dataFrame)
    covMatrix = calculateCovarianceMatrix(centeredDataFrame)
    diagonalCovMatrix = np.diag(covMatrix)
    sortedDiagonalCovMatrix = np.argsort(diagonalCovMatrix)[::-1]
    topFeatures_ = sortedDiagonalCovMatrix[:featureAmount]
    topFeaturesMatrix_ = dataset.iloc[:, topFeatures_]

    return topFeaturesMatrix_

def performRBFKernelPCA(dataset, desiredVarianceRatio):
    RBF = kernelRBF(dataset)
    centeredRBF = centerKernel(RBF)
    eigenvalues, eigenvectors = eigenDecomposition(centeredRBF)
    PCA, neededEigenvalues, VarianceRatio = performPCA(centeredRBF, desiredVarianceRatio, eigenvalues, eigenvectors)

    return centeredRBF, PCA, neededEigenvalues, VarianceRatio

def performPolyKernelPCA(dataset, desiredVarianceRatio):
    Poly = kernelPoly(dataset)
    centeredPoly = centerKernel(Poly)
    eigenvalues, eigenvectors = eigenDecomposition(centeredPoly)
    PCA, neededEigenvalues, VarianceRatio = performPCA(centeredPoly, desiredVarianceRatio, eigenvalues, eigenvectors)

    return centeredPoly, PCA, neededEigenvalues, VarianceRatio

def performLinearKernelPCA(dataset, desiredVarianceRatio):
    Linear = kernelLinear(dataset)
    centeredLinear = centerKernel(Linear)
    eigenvalues, eigenvectors = eigenDecomposition(centeredLinear)
    PCA, neededEigenvalues, VarianceRatio = performPCA(centeredLinear, desiredVarianceRatio, eigenvalues, eigenvectors)

    return centeredLinear, PCA, neededEigenvalues, VarianceRatio

def fullPCA(dataset, desiredVarianceRatio):
    centeredDataFrame = centerData(dataFrame)
    covMatrix = calculateCovarianceMatrix(centeredDataFrame)
    eigenvalue, eigenvector = eigenDecomposition(covMatrix)
    PCA, neededEigenvalues, VarianceRatio = performPCA(covMatrix, desiredVarianceRatio, eigenvalues, eigenvectors)

    return covMatrix, PCA, neededEigenvalues, VarianceRatio

In [ ]:
# Read Data
lungData = pd.read_csv("Lung.csv")

In [ ]:
# Part 1: Principal Component Analysis (PCA)
# 1.1 Implement PCA from Scratch:
# A. Compute the Covariance Matrix

# Center Dataset
centeredLungData = centerData(lungData)

# Calculate and Display the Covariance Matrix
covarianceMatrix = calculateCovarianceMatrix(centeredLungData)
print("Covariance Matrix:\n", covarianceMatrix)

Covariance Matrix:
 [[ 3.64132003e+07  3.63754207e+07  3.63903211e+07 ...  2.05735354e+05
   1.37170778e+07 -5.05024303e+01]
 [ 3.63754207e+07  3.63447280e+07  3.63557087e+07 ...  2.02655549e+05
   1.35841085e+07 -5.17544440e+01]
 [ 3.63903211e+07  3.63557087e+07  3.63742618e+07 ...  2.04431667e+05
   1.39865996e+07 -5.32241424e+01]
 ...
 [ 2.05735354e+05  2.02655549e+05  2.04431667e+05 ...  1.47418361e+05
   8.95987577e+05 -1.25884123e+01]
 [ 1.37170778e+07  1.35841085e+07  1.39865996e+07 ...  8.95987577e+05
   7.03172944e+08 -2.95837066e+02]
 [-5.05024303e+01 -5.17544440e+01 -5.32241424e+01 ... -1.25884123e+01
  -2.95837066e+02  7.30344184e-01]]


In [ ]:
# A. Compute the Eigenvalues and Eigenvectors
eigenvalue, eigenvector = eigenDecomposition(covarianceMatrix)
print("Sorted Eigenvalues:\n", eigenvalue)
print("\nSorted Eigenvector:\n", eigenvector)

Sorted Eigenvalues:
 [ 1.77291275e+10+0.j  6.06036261e+09+0.j  2.54877425e+09+0.j ...
 -2.96954821e-10+0.j -4.55695923e-10+0.j -3.51129529e-08+0.j]

Sorted Eigenvector:
 [[-2.36224092e-02+0.j -3.21901928e-02+0.j  4.51299988e-03+0.j ...
   2.28750554e-03+0.j  9.09863298e-03+0.j -2.97084335e-02+0.j]
 [-2.35450150e-02+0.j -3.22210286e-02+0.j  4.55744430e-03+0.j ...
   2.42228621e-03+0.j  9.16344813e-03+0.j -2.96883768e-02+0.j]
 [-2.36032857e-02+0.j -3.20342350e-02+0.j  4.52404866e-03+0.j ...
   2.35472139e-03+0.j  9.52760287e-03+0.j -2.96439024e-02+0.j]
 ...
 [-8.51728975e-04+0.j  6.27632888e-04+0.j -1.59468875e-03+0.j ...
   2.33676264e-03+0.j  9.25026628e-03+0.j -2.96765176e-02+0.j]
 [-5.12228931e-02+0.j  1.17124701e-01+0.j  1.72997865e-02+0.j ...
   2.35311817e-03+0.j  9.26154590e-03+0.j -2.96808682e-02+0.j]
 [ 4.73735208e-08+0.j -5.75140730e-09+0.j  5.20943857e-07+0.j ...
   2.24363107e-03+0.j  9.10400769e-03+0.j  1.63506550e-04+0.j]]


In [ ]:
# B. Apply PCA Implementation to Reduce the Dimensionality of the Features in the Data Frame
selectedEigenvectors = 2
lungPCAData = np.dot(centeredLungData, eigenvector[:, :selectedEigenvectors])

print("Centered Lung Data Shape:", centeredLungData.shape)
print("PCA Lung Data Shape:", lungPCAData.shape)

Centered Lung Data Shape: (1091, 1882)
PCA Lung Data Shape: (1091, 2)


In [ ]:
# C. Select an Appropriate Number of Principal Components to Retain A Significant Amount of Variance
desiredVarianceRatio = 0.98
lungPCAData, neededEigenvalues, calculatedVarianceRatio = performPCA(centeredLungData, desiredVarianceRatio, eigenvalue, eigenvector)

# Display Shape
print("Centered Lung Data Shape:", centeredLungData.shape)
print("PCA Lung Data Shape:", lungPCAData.shape)

The Needed Amount of Eigenvalues: 12
Appropriate Variance Ratio: 0.9821908623824441
Centered Lung Data Shape: (1091, 1882)
PCA Lung Data Shape: (1091, 12)


In [ ]:
# 1.2 PCA using scikit-learn:
# A. Use the PCA Module from sklearn to Perfrom Dimensionality Reduction on the Dataset
sklearnPCA = PCA(n_components=0.98)
reducedLungData = sklearnPCA.fit_transform(centeredLungData)

# Display Differences in Shape
print("Original Lung Data Shape:", lungData.shape)
print("Lung Data Shape with sklearn:", reducedLungData.shape)

# Display Variance Ratio
sklearnVarianceRatio = sklearnPCA.explained_variance_ratio_
print("\nVariance Ratios with sklearn\n", sklearnVarianceRatio)

# Display Principal Components
sklearnPrincipalComponents = sklearnPCA.components_
print("\nPricipal Components with sklearn\n", sklearnPrincipalComponents)

Original Lung Data Shape: (1091, 1882)
Lung Data Shape with sklearn: (1091, 12)

Variance Ratios with sklearn
 [0.53479798 0.18281045 0.07688361 0.04422609 0.03984111 0.0321145
 0.02158253 0.01672605 0.01564304 0.00807935 0.00582232 0.00366384]

Pricipal Components with sklearn
 [[-2.36224092e-02 -2.35450150e-02 -2.36032857e-02 ... -8.51728975e-04
  -5.12228931e-02  4.73735208e-08]
 [ 3.21901928e-02  3.22210286e-02  3.20342350e-02 ... -6.27632888e-04
  -1.17124701e-01  5.75140730e-09]
 [ 4.51299988e-03  4.55744430e-03  4.52404866e-03 ... -1.59468875e-03
   1.72997865e-02  5.20943857e-07]
 ...
 [ 2.29352854e-03  2.54689541e-03  2.52296340e-03 ...  1.26622531e-03
  -1.73913784e-01  2.72098241e-07]
 [-2.02407724e-01 -2.02409532e-01 -2.02728766e-01 ... -1.30647328e-04
   4.76300628e-01  1.42455879e-06]
 [-1.20476790e-01 -1.20497462e-01 -1.20944036e-01 ...  1.88024080e-03
   1.19152416e-01 -1.71007889e-06]]


In [ ]:
# B. Compare the From-Scratch Implementation with the PCA
# Module Compare Variance and the reduced feature set.
# Total Variance Ratio
totalSklearnVariance = np.sum(sklearnVarianceRatio)
print("From-Scratch Vs. Sklearn")
print("Total From-Scratch Variance:", calculatedVarianceRatio)
print("Total Sklearn Variance:", totalSklearnVariance)

# Number of Principal Components
print("\nTotal From-Scratch Principal Components:", neededEigenvalues)
print("Total Sklearn Principal Components:", sklearnPrincipalComponents.shape[0])

# Number of Components
totalComponents = np.size(lungPCAData)
totalSklearnComponents = np.size(reducedLungData)
print("\nTotal From-Scratch Components:", totalComponents)
print("Total Sklearn Components:", totalSklearnComponents)

# Shape
print("\nFrom-Scratch Shape:", lungPCAData.shape)
print("Sklearn Shape:", reducedLungData.shape)

From-Scratch Vs. Sklearn
Total From-Scratch Variance: 0.9821908623824441
Total Sklearn Variance: 0.9821908623824441

Total From-Scratch Principal Components: 12
Total Sklearn Principal Components: 12

Total From-Scratch Components: 13092
Total Sklearn Components: 13092

From-Scratch Shape: (1091, 12)
Sklearn Shape: (1091, 12)


In [ ]:
# Part 2: Kernel PCA (KPCA)
# 2.1 KPCA with RBF Kernel:
# A. Implement Kernel PCA with the Radial Basis Function (RBF) Kernel from Scratch.
kernelMatrixRBF = kernelRBF(lungData)
print("RBF Kernel Matrix:\n", kernelMatrixRBF)

RBF Kernel Matrix:
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
# B. Apply your KPCA Implementation to the Dataset.
# Center Dataset
centeredMatrixRBF = centerKernel(kernelMatrixRBF)
print("Centered RBF Kernel Matrix:\n", centeredMatrixRBF)
print("\nCentered RBF Kernel Matrix Shape:", centeredMatrixRBF.shape)

# Compute and Sort the Eigenvalues and Eigenvectors
eigenvaluesRBF, eigenvectorsRBF = eigenDecomposition(centeredMatrixRBF)

# Apply PCA to the RBF Kernel
kpcaRBF, neededEigenvaluesRBF, VarRatioRBF = performPCA(centeredMatrixRBF, desiredVarianceRatio, eigenvaluesRBF, eigenvectorsRBF)
print("RBF KPCA Lung Data Shape:", kpcaRBF.shape)

Centered RBF Kernel Matrix:
 [[ 9.99083410e-01 -9.16590284e-04 -9.16590284e-04 ... -9.16590284e-04
  -9.16590284e-04 -9.16590284e-04]
 [-9.16590284e-04  9.99083410e-01 -9.16590284e-04 ... -9.16590284e-04
  -9.16590284e-04 -9.16590284e-04]
 [-9.16590284e-04 -9.16590284e-04  9.99083410e-01 ... -9.16590284e-04
  -9.16590284e-04 -9.16590284e-04]
 ...
 [-9.16590284e-04 -9.16590284e-04 -9.16590284e-04 ...  9.99083410e-01
  -9.16590284e-04 -9.16590284e-04]
 [-9.16590284e-04 -9.16590284e-04 -9.16590284e-04 ... -9.16590284e-04
   9.99083410e-01 -9.16590284e-04]
 [-9.16590284e-04 -9.16590284e-04 -9.16590284e-04 ... -9.16590284e-04
  -9.16590284e-04  9.99083410e-01]]

Centered RBF Kernel Matrix Shape: (1091, 1091)
The Needed Amount of Eigenvalues: 1069
Appropriate Variance Ratio: 0.9807339449541213
RBF KPCA Lung Data Shape: (1091, 1069)


In [ ]:
# 2.2 KPCA with Polynomial Kernel:
# A. Implement Kernel PCA with a Polynomial Kernel from Scratch.
# Initialize Kernel
kernelMatrixPoly = kernelPoly(lungData)
print("Polynomial Kernel Matrix:\n", kernelMatrixPoly)

Polynomial Kernel Matrix:
 [[8.21982682e+24 4.62113325e+24 1.95915586e+24 ... 2.84024740e+24
  2.26710834e+24 1.93601609e+24]
 [4.62113325e+24 2.72918021e+24 1.21895330e+24 ... 1.66226238e+24
  1.37885868e+24 1.18547601e+24]
 [1.95915586e+24 1.21895330e+24 6.28469471e+23 ... 7.45057458e+23
  6.48915781e+23 5.77310621e+23]
 ...
 [2.84024740e+24 1.66226238e+24 7.45057458e+23 ... 1.05619127e+24
  8.45436977e+23 7.33697885e+23]
 [2.26710834e+24 1.37885868e+24 6.48915781e+23 ... 8.45436977e+23
  8.55881991e+23 6.50137960e+23]
 [1.93601609e+24 1.18547601e+24 5.77310621e+23 ... 7.33697885e+23
  6.50137960e+23 5.59631138e+23]]


In [ ]:
# B. Apply your KPCA Implementation to the Dataset.
# Center Dataset
centeredMatrixPoly = centerKernel(kernelMatrixPoly)
print("Centered Polynomial Kernel Matrix:\n", centeredMatrixPoly)
print("\nCentered Polynomial Kernel Matrix Shape:", centeredMatrixPoly.shape)

# Compute and Sort the Eigenvalues and Eigenvectors
eigenvaluesPoly, eigenvectorsPoly = eigenDecomposition(centeredMatrixPoly)

# Apply PCA to the Polynomial Kernel
kpcaPoly, neededEigenvaluesPoly, VarRatioPoly = performPCA(centeredMatrixPoly, desiredVarianceRatio, eigenvaluesPoly, eigenvectorsPoly)
print("Polynomial KPCA Lung Data Shape:", kpcaPoly.shape)

Centered Polynomial Kernel Matrix:
 [[ 4.86363915e+24  2.05052781e+24  5.65342488e+21 ...  7.04501688e+23
   2.15384198e+23 -9.17955634e+21]
 [ 2.05052781e+24  9.44157011e+23  5.10330925e+22 ...  3.12098906e+23
   1.12716768e+23  2.58625988e+22]
 [ 5.65342488e+21  5.10330925e+22  7.76522612e+22 ...  1.19969769e+22
  -1.23134919e+20  3.48002059e+22]
 ...
 [ 7.04501688e+23  3.12098906e+23  1.19969769e+22 ...  1.40887522e+23
   1.41547898e+22  8.94419847e+21]
 [ 2.15384198e+23  1.12716768e+23 -1.23134919e+20 ...  1.41547898e+22
   1.08621369e+23  9.40583918e+21]
 [-9.17955634e+21  2.58625988e+22  3.48002059e+22 ...  8.94419847e+21
   9.40583918e+21  2.54275181e+22]]

Centered Polynomial Kernel Matrix Shape: (1091, 1091)
The Needed Amount of Eigenvalues: 22
Appropriate Variance Ratio: 0.9805665858548569
Polynomial KPCA Lung Data Shape: (1091, 22)


In [ ]:
# 2.3 KPCA with Linear Kernel:
# A. Implement Kernel PCA with a Linear Kernel from Scratch.
# Create the Linear Kernel
kernelMatrixLinear = kernelLinear(lungData)
print("Linear Kernel Matrix:\n", kernelMatrixLinear)

Linear Kernel Matrix:
 [[3.79816511e+11 3.13474665e+11 2.35491885e+11 ... 2.66525238e+11
  2.47235240e+11 2.34561071e+11]
 [3.13474665e+11 2.63004808e+11 2.01039766e+11 ... 2.22939047e+11
  2.09472107e+11 1.99182206e+11]
 [2.35491885e+11 2.01039766e+11 1.61205929e+11 ... 1.70614604e+11
  1.62935503e+11 1.56707385e+11]
 ...
 [2.66525238e+11 2.22939047e+11 1.70614604e+11 ... 1.91661025e+11
  1.77956297e+11 1.69743064e+11]
 [2.47235240e+11 2.09472107e+11 1.62935503e+11 ... 1.77956297e+11
  1.78686158e+11 1.63037731e+11]
 [2.34561071e+11 1.99182206e+11 1.56707385e+11 ... 1.69743064e+11
  1.63037731e+11 1.55091110e+11]]


In [ ]:
# B. Apply your KPCA Implementation to the Dataset.
# Center Dataset
centeredMatrixLinear = centerKernel(kernelMatrixLinear)
print("Centered Linear Kernel Matrix:\n", centeredMatrixLinear)
print("\nCentered Linear Kernel Matrix Shape:", centeredMatrixLinear.shape)

# Compute and Sort the Eigenvalues and Eigenvectors
eigenvaluesLinear, eigenvectorsLinear = eigenDecomposition(centeredMatrixLinear)

# Apply PCA to the Linear Kernel
kpcaLinear, neededEigenvaluesLinear, VarRatioLinear = performPCA(centeredMatrixLinear, desiredVarianceRatio, eigenvaluesLinear, eigenvectorsLinear)
print("Linear KPCA Lung Data Shape:", kpcaLinear.shape)

Centered Linear Kernel Matrix:
 [[8.64994020e+10 5.21882661e+10 1.25151964e+10 ... 3.10442219e+10
  1.53306856e+10 1.22554700e+10]
 [5.21882661e+10 3.37491183e+10 1.00937877e+10 ... 1.94887410e+10
  9.59826172e+09 8.90731446e+09]
 [1.25151964e+10 1.00937877e+10 8.56966080e+09 ... 5.47400868e+09
  1.37136880e+09 4.74220401e+09]
 ...
 [3.10442219e+10 1.94887410e+10 5.47400868e+09 ... 1.40161016e+10
  3.88783503e+09 5.27355508e+09]
 [1.53306856e+10 9.59826172e+09 1.37136880e+09 ... 3.88783503e+09
  8.19415686e+09 2.14468340e+09]
 [1.22554700e+10 8.90731446e+09 4.74220401e+09 ... 5.27355508e+09
  2.14468340e+09 3.79701605e+09]]

Centered Linear Kernel Matrix Shape: (1091, 1091)
The Needed Amount of Eigenvalues: 12
Appropriate Variance Ratio: 0.9821908623824412
Linear KPCA Lung Data Shape: (1091, 12)


In [ ]:
# 2.4 Combining Kernels:
# A. Combine Two Different Kernels (e.g., RBF and Polynomial) and Apply the Combined KPCA to the Dataset.
alpha = 0.5
beta = 0.5
combinedPolyRBF = weightedKernelSum(alpha, beta, centeredMatrixPoly, centeredMatrixRBF)
print("Polynomial RBF Kernel Matrix:\n", combinedPolyRBF)
print("\nPolynomial RBF Kernel Matrix Shape:", combinedPolyRBF.shape)

# Compute and Sort the Eigenvalues and Eigenvectors
eigenvaluesPolyRBF, eigenvectorsPolyRBF = eigenDecomposition(combinedPolyRBF)

# Apply PCA to the Combined Kernel
kpcaPolyRBF, neededEigenvaluesPolyRBF, VarRatioPolyRBF = performPCA(combinedPolyRBF, desiredVarianceRatio, eigenvaluesPolyRBF, eigenvectorsPolyRBF)
print("Combined KPCA Lung Data Shape:", kpcaPolyRBF.shape)

Polynomial RBF Kernel Matrix:
 [[ 2.43181958e+24  1.02526391e+24  2.82671244e+21 ...  3.52250844e+23
   1.07692099e+23 -4.58977817e+21]
 [ 1.02526391e+24  4.72078506e+23  2.55165463e+22 ...  1.56049453e+23
   5.63583839e+22  1.29312994e+22]
 [ 2.82671244e+21  2.55165463e+22  3.88261306e+22 ...  5.99848844e+21
  -6.15674593e+19  1.74001029e+22]
 ...
 [ 3.52250844e+23  1.56049453e+23  5.99848844e+21 ...  7.04437608e+22
   7.07739491e+21  4.47209923e+21]
 [ 1.07692099e+23  5.63583839e+22 -6.15674593e+19 ...  7.07739491e+21
   5.43106845e+22  4.70291959e+21]
 [-4.58977817e+21  1.29312994e+22  1.74001029e+22 ...  4.47209923e+21
   4.70291959e+21  1.27137590e+22]]

Polynomial RBF Kernel Matrix Shape: (1091, 1091)
The Needed Amount of Eigenvalues: 22
Appropriate Variance Ratio: 0.9805665858548569
Combined KPCA Lung Data Shape: (1091, 22)


In [ ]:
# B. Evaluate the Classification Performance Using Accuracy Metrics for the Combined Kernels.
newLungData = lungData.iloc[:,:-1].to_numpy()
labelLungData = lungData.iloc[:,-1].to_numpy()

In [ ]:
# Train and Test Data
xTrain, xTest, yTrain, yTest = train_test_split(newLungData,labelLungData,test_size=.02)

# Create the RBF Kernel
# Trained Data
xTrainRBF = kernelRBF(xTrain)
centeredxTrainRBF = centerKernel(xTrainRBF)

# Test Data
xTestRBF = crossKernelRBF(xTest, xTrain)
centeredxTestRBF = centerCrossKernel(xTestRBF, xTrainRBF)

# Display
print("Trained RBF Kernel Matrix Shape:", centeredxTrainRBF.shape)
print("Tested RBF Kernel Matrix Shape:", centeredxTestRBF.shape)

# Create the Polynomial Kernel
# Trained Data
xTrainPoly = kernelPoly(xTrain)
centeredxTrainPoly = centerKernel(xTrainPoly)

# Test Data
xTestPoly = crossKernelPoly(xTest, xTrain)
centeredxTestPoly = centerCrossKernel(xTestPoly, xTrainPoly)

# Display
print("\nTrained Polynominal Kernel Matrix Shape:", centeredxTrainPoly.shape)
print("Tested Polynominal Kernel Matrix Shape:", centeredxTestPoly.shape)

Trained RBF Kernel Matrix Shape: (1069, 1069)
Tested RBF Kernel Matrix Shape: (22, 1069)

Trained Polynominal Kernel Matrix Shape: (1069, 1069)
Tested Polynominal Kernel Matrix Shape: (22, 1069)


In [ ]:
# Combine the Data
xTrainCombinedKernel = weightedKernelSum(alpha, beta, centeredxTrainPoly, centeredxTrainRBF)
xTestCombinedKernel = weightedKernelSum(alpha, beta, centeredxTestPoly, centeredxTestRBF)
print("Trained Polynomial RBF Kernel Matrix Shape:", xTrainCombinedKernel.shape)
print("Test Polynomial RBF Kernel Matrix Shape:", xTestCombinedKernel.shape)

# Perform Eigendecomposition
xTrainEigenvalues, xTrainEigenvectors = eigenDecomposition(xTrainCombinedKernel)

# Apply PCA
xTrainCombinedKPCA, xTrainNeededEigenvalues, xTrainVarRatio = performPCA(xTrainCombinedKernel, desiredVarianceRatio, xTrainEigenvalues, xTrainEigenvectors)
print("Combined KPCA xTrain Shape:", xTrainCombinedKPCA.shape)

# Apply Test Data Transform
xTestTransformedKernel = np.dot(xTestCombinedKernel, xTrainEigenvectors[:,:xTrainNeededEigenvalues])
print("Transformed Kernel xTest Shape:", xTestTransformedKernel.shape)

Trained Polynomial RBF Kernel Matrix Shape: (1069, 1069)
Test Polynomial RBF Kernel Matrix Shape: (22, 1069)
The Needed Amount of Eigenvalues: 22
Appropriate Variance Ratio: 0.9805523791625611
Combined KPCA xTrain Shape: (1069, 22)
Transformed Kernel xTest Shape: (22, 22)


In [ ]:
# Check Accuracy Metric
xTrainCombinedKPCA, xTestTransformedKernel, yTrain, yTest, yPredict = naiveGaussianClassifier(xTrainCombinedKPCA, xTestTransformedKernel, yTrain, yTest)
accuracyScore = accuracyMetric(yTest, yPredict)
print("Accuracy Metric:", accuracyScore)

Accuracy Metric: 50.0


In [ ]:
# Part 3: Testing and Evaluation
# 3.1 Preparing the Data:
# A. Split the Lung Data Frame into Train and Test Datasets
# Train and Test Data
xTrainNew, xTestNew, yTrainNew, yTestNew = train_test_split(newLungData,labelLungData,test_size=.02)

In [ ]:
# B. Use the PCA and KPCA models (RBF, Polynomial, Linear, and combined kernels)
# trained on the Train dataset to transform the Test dataset.
# Kernels
sklearnRBF = KernelPCA(n_components=5, kernel='rbf')
sklearnPoly = KernelPCA(n_components=5, kernel='poly')
sklearnLinear = KernelPCA(n_components=5, kernel='linear')

# C. Ensure the Dimensionality Reduction is Consistent with the Training Data.
# RBF
xTrainReducedRBF = sklearnRBF.fit_transform(xTrainNew)
xTestReducedRBF = sklearnRBF.transform(xTestNew)
print("RBF Kernel")
print("RBF X Train Reduced Shape:", xTrainReducedRBF.shape)
print("RBF X Test Reduced Shape:", xTestReducedRBF.shape)

# Poly
xTrainReducedPoly = sklearnPoly.fit_transform(xTrainNew)
xTestReducedPoly = sklearnPoly.transform(xTestNew)
print("\nPolynomial Kernel")
print("Polynomial X Train Reduced Shape:", xTrainReducedPoly.shape)
print("Polynomial X Test Reduced Shape:", xTestReducedPoly.shape)

# Linear
xTrainReducedLinear = sklearnLinear.fit_transform(xTrainNew)
xTestReducedLinear = sklearnLinear.transform(xTestNew)
print("\nLinear Kernel")
print("Linear X Train Reduced Shape:", xTrainReducedLinear.shape)
print("Linear X Test Reduced Shape:", xTestReducedLinear.shape)

# Combined RBF and Polynomial
xTrainReducedCombined = weightedKernelSum(alpha, beta, xTrainReducedRBF, xTrainReducedPoly)
xTestReducedCombined = weightedKernelSum(alpha, beta, xTestReducedRBF, xTestReducedPoly)

# Combined RBF and Polynomial
print("\nCombined RBF and Poly Kernel")
print("Combined RBF and Poly X Train Reduced Shape:", xTrainReducedCombined.shape)
print("Combined RBF and Poly X Test Reduced Shape:", xTestReducedCombined.shape)

RBF Kernel
RBF X Train Reduced Shape: (1069, 5)
RBF X Test Reduced Shape: (22, 5)

Polynomial Kernel
Polynomial X Train Reduced Shape: (1069, 5)
Polynomial X Test Reduced Shape: (22, 5)

Linear Kernel
Linear X Train Reduced Shape: (1069, 5)
Linear X Test Reduced Shape: (22, 5)

Combined RBF and Poly Kernel
Combined RBF and Poly X Train Reduced Shape: (1069, 5)
Combined RBF and Poly X Test Reduced Shape: (22, 5)


In [ ]:
# 3.2 Covariance Matrix Analysis:
# A. Calculate the Covariance Matrix of the Dataset.
print("Covariance Matrix:\n", covarianceMatrix)
print("\nCovariance Matrix Shape:", covarianceMatrix.shape)

Covariance Matrix:
 [[ 3.64132003e+07  3.63754207e+07  3.63903211e+07 ...  2.05735354e+05
   1.37170778e+07 -5.05024303e+01]
 [ 3.63754207e+07  3.63447280e+07  3.63557087e+07 ...  2.02655549e+05
   1.35841085e+07 -5.17544440e+01]
 [ 3.63903211e+07  3.63557087e+07  3.63742618e+07 ...  2.04431667e+05
   1.39865996e+07 -5.32241424e+01]
 ...
 [ 2.05735354e+05  2.02655549e+05  2.04431667e+05 ...  1.47418361e+05
   8.95987577e+05 -1.25884123e+01]
 [ 1.37170778e+07  1.35841085e+07  1.39865996e+07 ...  8.95987577e+05
   7.03172944e+08 -2.95837066e+02]
 [-5.05024303e+01 -5.17544440e+01 -5.32241424e+01 ... -1.25884123e+01
  -2.95837066e+02  7.30344184e-01]]

Covariance Matrix Shape: (1882, 1882)


In [ ]:
# B. Identify the Top 10 Features with the Highest Covariance Values.
diagonalCovarianceMatrix = np.diag(covarianceMatrix)
sortDiagonalCovMatrix = np.argsort(diagonalCovarianceMatrix)[::-1]
topFeatures = sortDiagonalCovMatrix[:10]

print("Top 10 Covariance Matrix Features:\n", topFeatures)
print("\nTop 10 Covariance Matrix Features Shape:", topFeatures.shape)

Top 10 Covariance Matrix Features:
 [ 285  189  595  200  273  305  352 1880   25  226]

Top 10 Covariance Matrix Features Shape: (10,)


In [ ]:
# C. Extract the Top 10 Features and Evaluate the Classification Performance Using Accuracy Metrics.
# Extract
topFeaturesMatrix = lungData.iloc[:, topFeatures]
print("Top Features Dataset shape:", topFeaturesMatrix.shape)

# Train and Test
newTopFeatures = topFeaturesMatrix
xTrainTop, xTestTop, yTrainTop, yTestTop = train_test_split(newTopFeatures,labelLungData,test_size=.02)

# KPCA
sklearnTopRBF = KernelPCA(n_components=5, kernel='rbf')
xTrainTopReducedRBF = sklearnTopRBF.fit_transform(xTrainTop)
xTestTopReducedRBF = sklearnTopRBF.transform(xTestTop)
print("RBF Kernel")
print("RBF X Train Reduced Shape:", xTrainTopReducedRBF.shape)
print("RBF X Test Reduced Shape:", xTestTopReducedRBF.shape)

# Check Accuracy Metric
xTrainTopReducedRBF, xTestTopReducedRBF, yTrainTop, yTestTop, yPredictTop = naiveGaussianClassifier(xTrainTopReducedRBF, xTestTopReducedRBF, yTrainTop, yTestTop)
topAccuracyScore = accuracyMetric(yTestTop, yPredictTop)
print("Accuracy Metric:", topAccuracyScore)

Top Features Dataset shape: (1091, 10)
RBF Kernel
RBF X Train Reduced Shape: (1069, 5)
RBF X Test Reduced Shape: (22, 5)
Accuracy Metric: 50.0


In [ ]:
# 3.3 Classification Experiment:
# A. Implement the Following Classifiers:
# B. Test the classifiers on:
# C. For each classifier and dimensionality reduction technique,
# find the best number of components that yield the highest classification accuracy.
# D. Plot accuracy versus the number of components for each classifier and dimensionality reduction technique.
# E. Evaluate the classification performance using accuracy metrics (accuracy, precision, recall)
# and compare the effectiveness of PCA features, KPCA features, and top 10 features.

In [ ]:
#  - Whole data


In [ ]:
#  - Data reduced by PCA


In [ ]:
#  - Data reduced by KPCA with RBF kernel
kdaSklearnRBF = KernelPCA(n_components=5, kernel='rbf')

#  - Minimum Distance Classifier (build from scratch)
xTrainMDC, xTestMDC, yTrainMDC, yTestMDC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Bayes Classifier (build from scratch)
xTrainBC, xTestBC, yTrainBC, yTestBC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Naive Bayes Classifier (using sklearn)
xTrainNBC, xTestNBC, yTrainNBC, yTestNBC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - KNN Classifier (using sklearn)
xTrainKNN, xTestKNN, yTrainKNN, yTestKNN = train_test_split(newLungData,labelLungData,test_size=.02)

#  - LDA Classifier (using sklearn)
xTrainLDA, xTestLDA, yTrainLDA, yTestLDA = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Kernel Discriminant Analysis (KDA) with RBF kernel


In [ ]:
#  - Data reduced by KPCA with Polynomial kernel
kdaSklearnPoly = KernelPCA(n_components=5, kernel='poly')

#  - Minimum Distance Classifier (build from scratch)
xTrainMDC, xTestMDC, yTrainMDC, yTestMDC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Bayes Classifier (build from scratch)
xTrainBC, xTestBC, yTrainBC, yTestBC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Naive Bayes Classifier (using sklearn)
xTrainNBC, xTestNBC, yTrainNBC, yTestNBC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - KNN Classifier (using sklearn)
xTrainKNN, xTestKNN, yTrainKNN, yTestKNN = train_test_split(newLungData,labelLungData,test_size=.02)

#  - LDA Classifier (using sklearn)
xTrainLDA, xTestLDA, yTrainLDA, yTestLDA = train_test_split(newLungData,labelLungData,test_size=.02)


#  - Kernel Discriminant Analysis (KDA) with Polynomial kernels


In [ ]:
#  - Data reduced by KPCA with Linear kernel
kdaSklearnLinear = KernelPCA(n_components=5, kernel='linear')

#  - Minimum Distance Classifier (build from scratch)
xTrainMDC, xTestMDC, yTrainMDC, yTestMDC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Bayes Classifier (build from scratch)
xTrainBC, xTestBC, yTrainBC, yTestBC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - Naive Bayes Classifier (using sklearn)
xTrainNBC, xTestNBC, yTrainNBC, yTestNBC = train_test_split(newLungData,labelLungData,test_size=.02)

#  - KNN Classifier (using sklearn)
xTrainKNN, xTestKNN, yTrainKNN, yTestKNN = train_test_split(newLungData,labelLungData,test_size=.02)

#  - LDA Classifier (using sklearn)
xTrainLDA, xTestLDA, yTrainLDA, yTestLDA = train_test_split(newLungData,labelLungData,test_size=.02)


#  - Kernel Discriminant Analysis (KDA) with Linear kernel


In [ ]:
#  - Data reduced by the top 10 features(PCA and KPCA here by using SKLEARN)
